# Dynamic Convolutional Neural Network for sequential data

In [ ]:
import os
import sys
from pathlib import Path 

import numpy as np

import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from torchsummary import summary

import logging
logging.basicConfig(stream=sys.stdout, format='',
                level=logging.INFO, datefmt=None)
logger = logging.getLogger('elliptic_scouting')

from IPython.display import display, Markdown, HTML, Image

sys.path.append('../')
from utils import *
from evaluation.model_performance import *

torch.__version__


In [ ]:
#model 
KTOP = 2

def get_dmaxp(d, in_size, tot_L):
    m = (tot_L - in_size) * d // tot_L
    return max(KTOP, m)

def conv1dOShape(l_in, k, s=2, p=0):
    return (l_in - k + 2*p) // s + 1

class DCNN(nn.Module):
    def __init__(self, in_channels, d, tot_l=2, wide=False):
        super(DCNN, self).__init__()
        self._in_channels = in_channels
        self._d = d
        self._k = self._d + 3 if wide else 3 # input size s^{m x d} d number of features
        self._tot_l = tot_l
        
        self.model = nn.Sequential(
            nn.Conv1d(self._in_channels, 5, self._k, stride=2, dtype=torch.float),
            nn.LeakyReLU(),
            nn.MaxPool1d(get_dmaxp(d, conv1dOShape(self._d, self._k), self._tot_l)),
            nn.Conv1d(5, 3, 3, stride=2),
            nn.LeakyReLU(),
            nn.MaxPool1d(get_dmaxp(d, conv1dOShape(conv1dOShape(self._d, self._k), 3), self._tot_l)),
            nn.Flatten(0),
            nn.Linear(30, 1),
            nn.LeakyReLU(),
            nn.Sigmoid()
            )
        
    def forward(self, x):
        #x = torch.tensor(x, device='cpu', dtype=torch.float32)
        return self.model(x)
    

class EllipticDataset(Dataset):
    def __init__(self, X, y):
        self._X = X
        self._y = y

    def __len__(self):
        return len(self._X) 

    def __getitem__(self, idx):
        x = np.asarray(self._X.iloc[idx].values, dtype=np.float32)
        y = np.asarray(self._y.iloc[idx], dtype=np.float16)
        return torch.from_numpy(x), torch.from_numpy(y).type(torch.float)


In [ ]:
last_time_step = 49
last_train_time_step = 34
only_labeled = True

X_train_df, X_test_df, y_train, y_test = run_elliptic_preprocessing_pipeline(last_train_time_step=last_train_time_step,
                                                                             last_time_step=last_time_step,
                                                                             only_labeled=only_labeled)

In [ ]:
X_train_df.shape, y_train.shape, X_test_df.shape, y_test.shape

In [ ]:
BATCH_SIZE = 1
train_ds = EllipticDataset(X_train_df, y_train)
test_ds = EllipticDataset(X_test_df, y_test)
train_ds = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=False)
test_ds = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
for x, y in train_ds:
    print(x.shape)
    print(x.dtype)
    print(y.shape)
    print(y.dtype)
    break

In [ ]:
net = DCNN(BATCH_SIZE, X_train_df.shape[1])
net

In [ ]:
# conv1 = nn.Conv1d(1, 5, 3, stride=2)
# maxp1 = nn.MaxPool1d(get_dmaxp(d, conv1dOShape(d, 3), 2))
# conv2 = nn.Conv1d(5, 3, 3, stride=2)
# maxp2 = nn.MaxPool1d(get_dmaxp(d, conv1dOShape(conv1dOShape(d, 3), 3), 2))
# fl = nn.Flatten(0)
# ln = nn.Linear(3, 2)
# sm = nn.Softmax(dim=0)

# x1 = conv1(x)
# x2 = maxp1(x1)
# x3 = conv2(x2)
# x4 = maxp2(x3)
# x5 = fl(x4)
# x6 = ln(x5)
# out = sm(x6)
# out

In [ ]:
for x, y in train_ds:
    print(x)
    print(x.shape)
    print(type(x))
    print(net(x).view(-1))
    break

In [ ]:
EPOCHS = 5
ce_loss = torch.nn.BCELoss()
optimizer = torch.optim.Adam(params=net.parameters())

for x, y in train_ds:
    pred = net(x)
    ce_loss(pred, y)
    break

In [ ]:
EPOCHS = 5
ce_loss = torch.nn.BCELoss()
optimizer = torch.optim.Adam(params=net.parameters())

for epoch in range(EPOCHS):
    for x, y in train_ds:
        net.zero_grad()
        y_pred = net(x)
        loss = ce_loss(y_pred, y)
        # calculate gradients of loss with respect model params
        loss.backward()
        # update params
        optimizer.step()
    
    print(f'epoch {epoch} loss {loss.item()}')

In [ ]:
losses = []
for x, y in test_ds:
    pred = net(x)
    loss = ce_loss(pred, y)
    losses.append(loss.item())

In [ ]:
np.sum(np.asarray(losses)) / len(y_test)